In [28]:
import torch 
from d2l import torch as d2l

In [29]:
flag = True
devices=  d2l.try_all_gpus() if flag else d2l.try_gpu()#第一个返回device列表，第二个返回device
devices

[device(type='cuda', index=0),
 device(type='cuda', index=1),
 device(type='cuda', index=2),
 device(type='cuda', index=3)]

In [30]:
def run( X ):
    return [X.mm(X) for _ in range( 50 )]
x_base = torch.rand( size=( 4000 , 4000 ))
x_gpu1 = torch.rand( size=( 4000 , 4000 ) , device= devices[0] )
x_gpu2 = torch.rand( size=( 4000, 4000 ) ,device=devices[3] )

In [31]:
run( x_gpu1 )
run( x_gpu2 )
torch.cuda.synchronize()
with d2l.Benchmark( 'CPU time'):
    run( x_base )
    torch.cuda.synchronize()
with d2l.Benchmark( 'GPU1 time'):
    run( x_gpu1 )
    torch.cuda.synchronize()
with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    torch.cuda.synchronize()

CPU time: 5.4600 sec
GPU1 time: 0.1975 sec
GPU2 time: 0.0014 sec


In [32]:
with d2l.Benchmark('GPU1 & GPU2'):#异步导致gpu板子训练可以并行化
    run( x_gpu1 )
    run( x_gpu2 )
    torch.cuda.synchronize()

GPU1 & GPU2: 0.1864 sec


In [33]:
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

with d2l.Benchmark('在GPU1上运行'):
    y = run(x_gpu1)
    torch.cuda.synchronize()

with d2l.Benchmark('复制到CPU'):
    y_cpu = copy_to_cpu(y)
    torch.cuda.synchronize()


在GPU1上运行: 0.1862 sec
复制到CPU: 3.4440 sec


In [34]:
with d2l.Benchmark('在gpu1上运行并复制到cpu当中'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu( y )
    torch.cuda.synchronize()

在gpu1上运行并复制到cpu当中: 4.0524 sec


In [35]:
with d2l.Benchmark('在gpu1上运行并复制到cpu当中'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu( y , non_blocking=True )#在可以异步的地方我们采用异步执行。也就是说即使上一步没有计算完，我们可以将上一行已经计算好的一部分梯度送到cpu
    torch.cuda.synchronize()

在gpu1上运行并复制到cpu当中: 0.4422 sec


In [38]:
[d2l.try_gpu(i) for i in range(4)][0]


device(type='cuda', index=0)